# Design Pattern - Decoradores

## Decoradores
O design pattern de decoradores permite adicionar comportamento adicional para objetos específios. Uma forma de fazer isto é decorar o objeto (método, função, classe, com tipos que fornecem os comportamentos adicionais sem utilizar herança. Em Python especificamente, os decoradores são funções que aceitam outras funções (ou métodos) e retornam uma terceira função com o comportamento decorado. 

Decoradores na realidade são objetos que podem ser manipulados, se comportando com macros que podem ser aplicados para objetos, retornando outros objetos (normalmente, uma nova função). 

## Definindo um decorador
Para definir um decorador, é necessário ter um objeto manipulável (exemplo, uma função decoradora) que recebe outra função como paramêtro (função a ser decorada), retornando uma nova função (função decorada). O comportamento decorado demonstrado é apenas a inclusão de alguns prints, mas pode ser muito mais complexo, como lidar com atributos adicionais, pré-processar argumentos, etc. Como por exemplo:

In [3]:
# Decorador que encapsula a funcao original
def funcDecorador(func):
    # Nova funcao a ser implementada, decora a funcao original
    def funcDecorada():
        print("Chamando " + func.__name__ + "...")
        func() # Chamada da funcao original, qualquer outra linha eh comportamento decorado
        print(func.__name__ + " chamado...")
    return funcDecorada

## Utilizando decoradores
Podemos definir uma nova função para demonstrar um exemplo da utilização do decorador definido anteriormente.

In [6]:
def func():
    print("Função.")
    
func_dec = funcDecorador(func)
func_dec()

Chamando func...
Função.
func chamado...


O comportamento decorado demonstrado é muito simples, mas ele pode ser modificado para ser muito mais complexo. Observe que é possível utilizar o decorador sintático (diferente do design pattern) `@funcDecorador` para associar a função a ser decorada o decorador.

In [7]:
@funcDecorador
def func():
    print("Função.")
    
func_dec()

Chamando func...
Função.
func chamado...


## Funções com paramêtros
Decoradores podem ser aplicados para funções que possuem paramêtros, desde que o decorador também aceite os paramêtros.

## Decoradores Empilhados
Decoradores pode ser empilhados, ou seja, multiplos decoradores podem ser aplicado sobre o mesmo objeto. Para isto, é necessário garantir sempre que a quantidade de paramêtros do objeto decorado não se altere. 

In [15]:
def dec_negrito(func):
    def negrito():
        return "<b>" + func() + "</b>"
    return negrito

def dec_italico(func):
    def italico():
        return "<i>" + func() + "</i>"
    return italico

@dec_negrito
@dec_italico
def func_str():
    return 'Ola mundo.'

print(func_str())

<b><i>Ola mundo.</i></b>


## Decoradores Parametrizados
Decoradores também podem ser seus próprios paramêtros, entretanto, a sintaxe deles requer um tratamento especial. Por exemplo, para um paramêtro binário que define se a função deve ser chamada ou não, é necessário adicionar duas implementações diferentes dentro do decorador.

In [16]:
def func_par(executa = True):
    def decorador(func):
        def func_decorada():
            print("Tentando executar " + func.__name__ + "...")
            if(executa):
                print("Executando...")
                func()
            else:
                print("Não executado...")
        return func_decorada
    return decorador

@func_par()
def func1():
    print("Funcao 1.")
    
@func_par(executa = False)
def func2():
    print("Funcao 2.")

func1()
func2()

Tentando executar func1...
Executando...
Funcao 1.
Tentando executar func2...
Não executado...


## Decoradores de métodos
Até então, implementamos decoradores para funções normais, entretanto, é possível implementar decoradores para métodos também. É valido relembrar que métodos sempre recebem o paramêtro self, logo, é necessário que o decorador também receba este paramêtro. No exemplo abaixo, implementamos uma função decoradora de métodos. Note que a função definida dentro do decorador recebe o argumento self, já que a refêrencia ao método é feito de forma direta, e não através do objeto.

In [5]:
def decorador(metodo):
    def metodo_decorado(self):
        return "<b>" + metodo(self) + "</b>"
    return metodo_decorado
    
class Pessoa():
    def __init__(self,nome,idade):
        self.nome = nome
        self.idade = idade
     
    @decorador
    def get_nome_bold(self):
        return self.nome
    
x = Pessoa("Joao",18)
print(x.get_nome_bold())

<b>Joao</b>


Para métodos com paramêtros, basta passar os paramêtros do método após o self, da mesma forma que se faz na definição do método dentro da classe. Note que o método decorado deve chamar o método original também passando os paramêtros.

In [7]:
def decorador(metodo):
    def metodo_decorado(self,antecedente,consequente):
        print("Chamando o método...")
        return metodo(self,antecedente,consequente)
    return metodo_decorado
    
class Pessoa():
    def __init__(self,nome,idade):
        self.nome = nome
        self.idade = idade
     
    @decorador
    def conc_nome(self,antecedente,consequente):
        return antecedente + self.nome + consequente
    
x = Pessoa("Joao",18)
print(x.conc_nome("Ola ", ", seja bem vindo!"))

Chamando o método...
Ola Joao, seja bem vindo!


## Decoradores de classes
Como já comentado anteriormente, é possível decorar classes também. Isto pode ser feito para extender funcionalidades das classes, sem ter que realizar a herança. Por exemplo, há um design pattern chamado de singleton DP, cujo objetivo é permitir no máximo uma instância de uma classe no programa. Para implementar isto, pode se decorar uma classe.

In [8]:
def decorador_singleton(cls):
    instance = None
    
    def get_instance():
        nonlocal instance
        if instance is None:
            instance = cls()
        return instance
    return get_instance

In [9]:
@decorador_singleton
class Pessoa():
    def print_self(self):
        print(self)

Então, instânciamos dois objetos para a classe. Note que, após imprimir o objeto, e criar um novo, o endereço do novo não é armazenado, já que o mesmo teria que sobreescrever a váriavel instance, algo que é impossibilitado pelo if. 

In [9]:
p1 = Pessoa()
p1.print_self()
p2 = Pessoa()
p2.print_self()

## Execução de um decorador
A execução de um decorador geralmente ocorre em tempo de importação (quando o módulo é importado pelo Python). A função decorada e função original só são executadas entretanto quando ocorre uma chamada, no caso, em tempo de execução. 

## Functools Wrap
Ao utilizar um decorador, acaba se perdendo as informações (name, doc e module) da função,já que a que é retornada ao chamar o método é a da função decorada. Para contornar este problema, o Python possuí o pacote `functools`, que possuí o módulo `wraps`, que contém o decorador `@wraps`. O `@wraps` pode ser utilizado para atualizar os atributos da função decorada para aqueles da função original.

In [13]:
from functools import wraps

def decorador(func):
    @wraps(func)
    def func_decorada():
        """Funcao ja decorada."""
        print("Chamando funcao...")
        func()
    return func_decorada

@decorador
def funcao():
    """Funcao a ser decorada."""
    print("hello world.")

funcao()
print(funcao.__doc__)

Chamando funcao...
hello world.
Funcao ja decorada.
